In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install colabgymrender==1.0.2

In [ ]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/

In [ ]:
import gym
from colabgymrender.recorder import Recorder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
env = gym.make("MountainCar-v0")
env._max_episode_steps = 1000
#####saved model parameters#####
# 48,24,3
#adam(lr=0.001)  mse

def create_model():
  inputs = layers.Input(shape=(2))# - actual used for saved mode
  layer1 = layers.Dense(48,activation='relu')(inputs)#
  layer2 = layers.Dense(24,activation='relu')(layer1)
  action = layers.Dense(3)(layer2)

  return keras.Model(inputs=inputs, outputs=action)

model = create_model()
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),loss="mean_squared_error")
target_model = create_model()
target_model.compile(optimizer=keras.optimizers.Adam(lr=0.001),loss="mean_squared_error")
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_6 (Dense)              (None, 48)                144       
_________________________________________________________________
dense_7 (Dense)              (None, 24)                1176      
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 75        
Total params: 1,395
Trainable params: 1,395
Non-trainable params: 0
_________________________________________________________________


In [ ]:

total_rewards = 0
frame_count = 0
episode_count = 0
max_memory_length = 100000
update_after = 4
update_target_after = 1000
epsilon = 1
gamma = 0.99

action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
flag=0
for i in range(1000):
  state = env.reset()
  state = np.array(state)
  rewards=0

  for step in range(1,1000):
    frame_count+=1

    if frame_count < 30000 or epsilon > np.random.rand(1)[0]:
      action = np.random.choice(3)
    else:
      state_tensor = tf.convert_to_tensor(state)
      state_tensor = tf.expand_dims(state_tensor,0)
      action_prob = model.predict(state_tensor)
      action = tf.argmax(action_prob[0]).numpy()

    epsilon -= 0.99/100000
    epsilon = max(epsilon,0.1)

    state_next,reward,done,info = env.step(action)
    state_next = np.array(state_next)

    rewards += reward

    action_history.append(action)
    state_history.append(state)
    state_next_history.append(state_next)
    done_history.append(done)
    rewards_history.append(reward)

    state = state_next

    if len(done_history) > 32:
      indices = np.random.choice(range(len(done_history)),size=32)

      state_sample = np.array([state_history[i] for i in indices])
      state_next_sample = np.array([state_next_history[i] for i in indices])
      rewards_sample = [rewards_history[i] for i in indices]
      action_sample = [action_history[i] for i in indices]
      done_sample = tf.convert_to_tensor([float(done_history[i]) for i in indices])

      future_rewards = target_model.predict(state_next_sample)
      current_rewards = model.predict(state_sample)
      updated_q = current_rewards[:]

      for idx,terminal in enumerate(done_sample):
        if terminal:
          future_rewards[idx]=0.0
        updated_q[idx,action_sample[idx]] = rewards_sample[idx] + gamma*tf.reduce_max(future_rewards,axis=1)[idx]

      model.train_on_batch(state_sample,updated_q)

    if frame_count % update_target_after ==0:
      target_model.set_weights(model.get_weights())
      #print("reward",total_rewards,"episode:",episode_count,"frame:",frame_count)

    if len(rewards_history)>max_memory_length:
      del rewards_history[:1]
      del state_history[:1]
      del state_next_history[:1]
      del action_history[:1]
      del done_history[:1]

    if done:
      break

  print("episode reward:",rewards,"mean reward",total_rewards,"episode:",episode_count,"frame:",frame_count)
  episode_reward_history.append(rewards)
  if len(episode_reward_history)>100:
    del episode_reward_history[:1]
  episode_count+=1
  total_rewards = np.mean(episode_reward_history)

  if episode_count%100==0:
    model.save('/content/gdrive/My Drive/Reinforcement-Learning-project/mountaincar-{}.h5'.format(episode_count))
    model.save('mountaincar-{}.h5'.format(episode_count))

  if total_rewards > -110:
    print("solved at episode: ",episode_count)
    break
#env.play()

In [ ]:
directory = './video'
env = Recorder(env, directory)
actions = []
reward = []
for h in range(10):
  done=False
  obs=env.reset()
  obs=np.array(obs)
  reward=0
  while not done:
    obs = tf.convert_to_tensor(obs)
    obs = tf.expand_dims(obs,0)
    action = model.predict(obs)
    a = tf.argmax(action[0]).numpy()
    obs_,r,done,_ = env.step(a)
    obs=np.array(obs_)
    actions.append(a)
    reward+=r
  print(reward)
#env.play()